In [5]:
import pandas as pd
import numpy as np
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import MinMaxScaler

# ---- STEP 1: Load the EEG datasets ----
stressed_file = "STRESSED.csv"  # Change to actual filename
relaxed_file = "RELAXED.csv"    # Change to actual filename

df_stressed = pd.read_csv(stressed_file)
df_relaxed = pd.read_csv(relaxed_file)

# ---- STEP 2: Handle Missing Values ----
df_stressed.dropna(inplace=True)
df_relaxed.dropna(inplace=True)

# ---- STEP 3: Apply Bandpass Filter (0.5Hz - 50Hz) ----
def bandpass_filter(data, lowcut=0.5, highcut=50.0, fs=256, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return filtfilt(b, a, data)

# Convert EEG values to float (Fixing the TypeError)
df_stressed["EEG_Value"] = pd.to_numeric(df_stressed["EEG_Value"], errors="coerce")
df_relaxed["EEG_Value"] = pd.to_numeric(df_relaxed["EEG_Value"], errors="coerce")

# Drop NaN values after conversion (if any)
df_stressed.dropna(subset=["EEG_Value"], inplace=True)
df_relaxed.dropna(subset=["EEG_Value"], inplace=True)

# ---- STEP 4: Normalize EEG Values (0 to 1 scaling) ----
scaler = MinMaxScaler()
df_stressed["EEG_Value"] = scaler.fit_transform(df_stressed[["EEG_Value"]])
df_relaxed["EEG_Value"] = scaler.fit_transform(df_relaxed[["EEG_Value"]])

# ---- STEP 5: Save the cleaned datasets ----
df_stressed.to_csv("cleaned_stressed_eeg.csv", index=False)
df_relaxed.to_csv("cleaned_relaxed_eeg.csv", index=False)

print("\n✅ Preprocessing complete. Cleaned files saved as 'cleaned_stressed_eeg.csv' & 'cleaned_relaxed_eeg.csv'.")



✅ Preprocessing complete. Cleaned files saved as 'cleaned_stressed_eeg.csv' & 'cleaned_relaxed_eeg.csv'.


In [ ]:
import numpy as np
import pandas as pd
from scipy.signal import welch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# Load preprocessed EEG data
df_stressed = pd.read_csv("cleaned_stressed_eeg.csv")
df_relaxed = pd.read_csv("cleaned_relaxed_eeg.csv")

# Sampling frequency (adjust if needed)
FS = 250  # Hz (assumed)

# Define EEG frequency bands
FREQ_BANDS = {
    "Delta": (0.5, 4),
    "Theta": (4, 8),
    "Alpha": (8, 13),
    "Beta": (13, 30),
    "Gamma": (30, 45)
}

# Function to compute power in frequency bands
def compute_power(eeg_signal, fs, band):
    """Compute power of a specific frequency band using Welch's method."""
    nperseg = min(len(eeg_signal), fs)
    if nperseg < 2:  # Prevent Welch calculation errors
        return 0
    f, Pxx = welch(eeg_signal, fs=fs, nperseg=nperseg)
    low, high = band
    return np.sum(Pxx[(f >= low) & (f <= high)])

# Function to extract features from EEG data
def extract_features(df):
    features = []
    for _, row in df.iterrows():
        eeg_signal = row["EEG_Value"]  # Get EEG values
        
        # Ensure eeg_signal is a NumPy array
        if isinstance(eeg_signal, str):  # If stored as a string
            eeg_signal = np.array(eeg_signal.split(), dtype=float)
        elif isinstance(eeg_signal, (int, float)):  # If it's a single value (unlikely)
            eeg_signal = np.array([eeg_signal], dtype=float)
        elif isinstance(eeg_signal, list):  # If it's already a list
            eeg_signal = np.array(eeg_signal, dtype=float)

        # Compute frequency band power
        band_powers = {name: compute_power(eeg_signal, FS, band) for name, band in FREQ_BANDS.items()}
        
        # Statistical features
        mean_val = np.mean(eeg_signal)
        std_dev = np.std(eeg_signal)
        
        # Store extracted features
        feature_dict = {
            "Mean": mean_val,
            "Std_Dev": std_dev,
            **band_powers,  # Unpack frequency band features
            "Label": row["Label"]
        }
        features.append(feature_dict)

    return pd.DataFrame(features)

# Extract features from both datasets
df_stressed_features = extract_features(df_stressed)
df_relaxed_features = extract_features(df_relaxed)

# Combine and save as a new dataset
df_combined = pd.concat([df_stressed_features, df_relaxed_features], ignore_index=True)
df_combined.to_csv("eeg_features_dataset.csv", index=False)

print("Feature extraction complete! ✅ EEG features saved to 'eeg_features_dataset.csv'.")

# Load extracted features
df = pd.read_csv("eeg_features_dataset.csv")

# Prepare data for ML models
X = df.drop(columns=["Label"])
y = df["Label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Train and evaluate models
models = {
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "SVM": SVC(kernel='linear', random_state=42),
    "KNN": KNeighborsClassifier(n_neighbors=5)
}

accuracies = {}
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    accuracies[name] = accuracy_score(y_test, y_pred)
    print(f"{name} Accuracy: {accuracies[name]:.4f}")

# Compare accuracy results
best_model = max(accuracies, key=accuracies.get)
print(f"Best performing model: {best_model} with accuracy {accuracies[best_model]:.4f}")


Feature extraction complete! ✅ EEG features saved to 'eeg_features_dataset.csv'.
Random Forest Accuracy: 0.9766
